In [1]:
import timeit
import neater_cnc_tableau as cnc
import numpy as np
import timeit
from random import choice
import chp

In [2]:
# apply random gate sequence with cnc sim:
def apply_sequence_of_clifford(simulator, sequence):
    """Apply a sequence of Clifford gates to the simulator."""
    for gate in sequence:
        if gate.startswith('h'):
            qubit = int(gate.split('_')[1])
            simulator.apply_hadamard(qubit)
        elif gate.startswith('s'):
            qubit = int(gate.split('_')[1])
            simulator.apply_phase(qubit)
        elif gate.startswith('cnot'):
            control, target = map(int, gate.split('_')[1:])
            simulator.apply_cnot(control, target)

# boolean function using GHZ state

In [10]:
# by default our initial state is |+>\otimes n:
def ghz_state_prep(simulator,n):
    for m in range(1,n):
        simulator.apply_hadamard(m)
    for m in range(1,n):
        simulator.apply_cnot(0,m)


In [37]:
def measurement_bases(n):
    # Initialize empty 3D array
    bases_array = np.empty((0, n, 2*n), dtype=int)

    # Create the first measurement basis
    zero_matrix = np.zeros((n, n), dtype=int)
    identity_matrix = np.eye(n, dtype=int)

    # Stack and expand the dimension to make it 3D
    first_basis = np.hstack((identity_matrix, zero_matrix))[np.newaxis, :, :]

    # initialize empty row vector:
    yint = np.zeros((1,2*n),dtype=int)

    # modify first row:
    y1 = yint.copy()
    y1[0,n] = 1
    y1[0,0] = 1

    # Concatenate along the first axis
    bases_array = np.concatenate((bases_array, first_basis), axis=0)

    # generate remaining measurements:
    for k in range(1,n):
        # initialize measurement bases:
        basis = first_basis.copy()
        basis[0,0,:] = y1

        # modify kth row:
        yk = yint.copy()
        yk[0,k] = 1
        yk[0,k+n] = 1
        basis[0,k,:] = yk

        # Concatenate along the first axis
        bases_array = np.concatenate((bases_array, basis), axis=0)
        

    return bases_array

In [72]:
def ghz_function(n):
    boolean_mapping = dict()
    for k in range(n):
        if k == 0:
            image = 0
        else:
            image = 1
        
        boolean_mapping[k] = image
    return boolean_mapping

In [78]:
# check results for range of qubit values:
for n in range(2,50): 
    # fix n-qubits:
    m = 1

    # initialize measurements:
    measurements = measurement_bases(n)

    # initialize empty list for outcomes:
    outcomes_array = []
    for i in range(n):

        # initialize simulator:
        simulator = cnc.CncSimulator(n,m)

        # prepare ghz state
        ghz_state_prep(simulator,n)

        bases = measurements[i,:,:]
        outcomes = []
        for j in range(n):
            outcome = simulator.measure(bases[j, :])
            outcomes.append(outcome)
        outcomes_array.append(outcomes)

    # check results:
    mapping = ghz_function(n)
    xor_outcomes = [sum(outcomes_array[k]) % 2 for k in range(n)]
    bool_list = [mapping[k] == xor_outcomes[k] for k in range(n)]
    # print results
    print(f"Simulation for {n} qubits gives correct result:", all(bool_list),"\n")

Simulation for 2 qubits gives correct result: True 

Simulation for 3 qubits gives correct result: True 

Simulation for 4 qubits gives correct result: True 

Simulation for 5 qubits gives correct result: True 

Simulation for 6 qubits gives correct result: True 

Simulation for 7 qubits gives correct result: True 

Simulation for 8 qubits gives correct result: True 

Simulation for 9 qubits gives correct result: True 

Simulation for 10 qubits gives correct result: True 

Simulation for 11 qubits gives correct result: True 

Simulation for 12 qubits gives correct result: True 

Simulation for 13 qubits gives correct result: True 

Simulation for 14 qubits gives correct result: True 

Simulation for 15 qubits gives correct result: True 

Simulation for 16 qubits gives correct result: True 

Simulation for 17 qubits gives correct result: True 

Simulation for 18 qubits gives correct result: True 

Simulation for 19 qubits gives correct result: True 

Simulation for 20 qubits gives corre